In [ ]:
import pandas as pd
import plotly.graph_objects as go
import os

# --- Configuration ---
# Choose a skill you want to analyze from your results folder
SKILL_TO_ANALYZE = 'python' 

# --- File Paths ---
forecast_path = f'../results/{SKILL_TO_ANALYZE}_forecast.csv'
history_path = f'../data/processed/timeseries/{SKILL_TO_ANALYZE}_timeseries.csv'

# --- Load Data ---
try:
    # Load the forecast data generated by Prophet
    forecast_df = pd.read_csv(forecast_path)
    forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
    
    # Load the original historical data to compare
    history_df = pd.read_csv(history_path)
    history_df['ds'] = pd.to_datetime(history_df['ds'])
    
    # Merge historical data into the forecast dataframe for plotting
    merged_df = forecast_df.merge(history_df, on='ds', how='left')
    
except FileNotFoundError as e:
    print(f"Error: Could not find a file. Make sure your paths are correct.\n{e}")
    # Stop execution if files are not found
    exit()

# --- Create Visualization ---
fig = go.Figure()

# Add the uncertainty interval (the shaded blue area)
fig.add_trace(go.Scatter(
    x=merged_df['ds'],
    y=merged_df['yhat_upper'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(68, 137, 238, 0.2)',
    hoverinfo='none',
    showlegend=False
))
fig.add_trace(go.Scatter(
    name='Uncertainty',
    x=merged_df['ds'],
    y=merged_df['yhat_lower'],
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(68, 137, 238, 0.2)',
    hoverinfo='none',
    showlegend=True
))

# Add the forecast line (the solid blue line)
fig.add_trace(go.Scatter(
    name='Forecast (yhat)',
    x=merged_df['ds'],
    y=merged_df['yhat'],
    mode='lines',
    line=dict(color='rgb(68, 137, 238)', width=3)
))

# Add the actual historical data points (the black dots)
fig.add_trace(go.Scatter(
    name='Actual Data (y)',
    x=merged_df['ds'],
    y=merged_df['y'],
    mode='markers',
    marker=dict(color='black', size=4)
))

# --- Customize Layout ---
fig.update_layout(
    title=f'Forecast for "{SKILL_TO_ANALYZE.title()}" Skill Demand',
    xaxis_title='Date',
    yaxis_title='Daily Mentions in Job Postings',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()